In [ ]:
#----train----
import os
import cv2
import gc
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from IPython.display import FileLink

# --- 1. LIBRARIES & INSTALLATION ---
try:
    import segmentation_models_pytorch as smp
except:
    os.system('pip install -q segmentation_models_pytorch albumentations')
    import segmentation_models_pytorch as smp

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset, DataLoader
from torch.amp import GradScaler, autocast

# --- 2. CONFIGURATION (Fixed for OOM) ---
CFG = {
    "img_size": 768,        
    "batch_size": 2,        # 4 se 2 kiya OOM bachane ke liye
    "grad_accum": 2,        # 2 batches ke baad update karega (Effective Batch Size = 4)
    "epochs": 40,           
    "patience": 8,          
    "lr": 1e-4,             
    "device": torch.device("cuda" if torch.cuda.is_available() else "cpu"),
    "train_img": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_images",
    "train_mask": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/train_masks",
    "test_dir": "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded",
    "model_path": "best_rugged_ultra.pth",
    "csv_path": "submission.csv",
    "threshold": 0.5        
}

# --- 3. AUGMENTATIONS ---
train_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.2),
    A.RandomRotate90(p=0.5),
    A.OneOf([
        A.GaussNoise(var_limit=(10.0, 50.0)),
        A.CLAHE(clip_limit=2.0),
        A.RandomBrightnessContrast(),
    ], p=0.4),
    A.Normalize(),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Resize(CFG["img_size"], CFG["img_size"]),
    A.Normalize(),
    ToTensorV2()
])

# --- 4. DATASET & LOSS ---
class OffroadDataset(Dataset):
    def __init__(self, images, img_dir, mask_dir, transform=None):
        self.images, self.img_dir, self.mask_dir, self.transform = images, img_dir, mask_dir, transform
    def __len__(self): return len(self.images)
    def __getitem__(self, idx):
        name = self.images[idx]
        img_path = os.path.join(self.img_dir, name)
        img = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)
        mask_name = os.path.splitext(name)[0] + ".png"
        mask = cv2.imread(os.path.join(self.mask_dir, mask_name), cv2.IMREAD_GRAYSCALE)
        mask = (mask > 0).astype(np.float32)
        if self.transform:
            aug = self.transform(image=img, mask=mask)
            img, mask = aug['image'], aug['mask']
        return img, mask.unsqueeze(0)

class RuggedLoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.dice = smp.losses.DiceLoss(mode='binary')
        self.focal = smp.losses.FocalLoss(mode='binary', gamma=2.0)
        self.bce = nn.BCEWithLogitsLoss()
    def forward(self, logits, targets):
        return 0.4 * self.dice(logits, targets) + 0.4 * self.focal(logits, targets) + 0.2 * self.bce(logits, targets)

def rle_encode(mask01):
    pixels = mask01.flatten(order="F") 
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

# --- 6. MAIN PIPELINE ---
def main():
    # Setup Data
    all_imgs = sorted([f for f in os.listdir(CFG["train_img"]) if f.endswith(('.png', '.jpg'))])
    train_idx, val_idx = train_test_split(all_imgs, test_size=0.1, random_state=42)

    if os.path.exists(CFG["model_path"]):
        print(f"✅ Skipping training phase...")
        train_needed = False
    else:
        print("🚀 Starting Ultra Training (EfficientNet-B6 for Memory Efficiency)...")
        train_needed = True

    # B6 is used instead of B7 to avoid OOM while maintaining accuracy
    model = smp.UnetPlusPlus(encoder_name="efficientnet-b6", encoder_weights="imagenet", in_channels=3, classes=1).to(CFG["device"])

    if train_needed:
        train_loader = DataLoader(OffroadDataset(train_idx, CFG["train_img"], CFG["train_mask"], train_transform), 
                                  batch_size=CFG["batch_size"], shuffle=True, num_workers=2, pin_memory=True)
        val_loader = DataLoader(OffroadDataset(val_idx, CFG["train_img"], CFG["train_mask"], val_transform), 
                                batch_size=CFG["batch_size"], shuffle=False, num_workers=2)

        optimizer = torch.optim.AdamW(model.parameters(), lr=CFG["lr"], weight_decay=1e-4)
        criterion = RuggedLoss()
        scaler = GradScaler()
        best_iou = 0
        patience_counter = 0

        for epoch in range(1, CFG["epochs"] + 1):
            model.train()
            pbar = tqdm(train_loader, desc=f"Epoch {epoch}")
            optimizer.zero_grad() # Move outside inner loop for accumulation
            
            for i, (imgs, masks) in enumerate(pbar):
                imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                
                with autocast(device_type='cuda'):
                    out = model(imgs)
                    loss = criterion(out, masks)
                    loss = loss / CFG["grad_accum"] # Normalize loss
                
                scaler.scale(loss).backward()
                
                if (i + 1) % CFG["grad_accum"] == 0:
                    scaler.step(optimizer)
                    scaler.update()
                    optimizer.zero_grad()
                
                del imgs, masks, out, loss
            
            # Validation
            model.eval()
            v_iou = 0
            with torch.no_grad():
                for imgs, masks in val_loader:
                    imgs, masks = imgs.to(CFG["device"]), masks.to(CFG["device"])
                    out = (torch.sigmoid(model(imgs)) > CFG["threshold"]).float()
                    inter = (out * masks).sum()
                    union = (out + masks).sum() - inter
                    v_iou += (inter + 1e-7) / (union + 1e-7)
            
            avg_iou = (v_iou / len(val_loader)).item()
            print(f"📊 Validation IoU: {avg_iou:.5f}")

            if avg_iou > best_iou:
                best_iou = avg_iou
                torch.save({'model_state_dict': model.state_dict(), 'iou': best_iou}, CFG["model_path"])
                patience_counter = 0
            else:
                patience_counter += 1
            
            gc.collect()
            torch.cuda.empty_cache()

            if patience_counter >= CFG["patience"]: break

    # --- INFERENCE ---
    print("\n🎬 Inference with TTA...")
    checkpoint = torch.load(CFG["model_path"], map_location=CFG["device"])
    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()
    
    test_imgs = sorted([f for f in os.listdir(CFG["test_dir"]) if f.endswith(('.png', '.jpg'))])
    results = []
    with torch.no_grad():
        for name in tqdm(test_imgs):
            img = cv2.cvtColor(cv2.imread(os.path.join(CFG["test_dir"], name)), cv2.COLOR_BGR2RGB)
            t1 = val_transform(image=img)["image"].unsqueeze(0).to(CFG["device"])
            t2 = val_transform(image=cv2.flip(img, 1))["image"].unsqueeze(0).to(CFG["device"])
            p1 = torch.sigmoid(model(t1))
            p2 = torch.flip(torch.sigmoid(model(t2)), [3])
            final_p = ((p1 + p2) / 2).cpu().numpy()[0, 0]
            mask01 = (cv2.resize(final_p, (960, 540), interpolation=cv2.INTER_NEAREST) > CFG["threshold"]).astype(np.uint8)
            results.append({"image_id": os.path.splitext(name)[0], "encoded_pixels": rle_encode(mask01)})
            if len(results) % 100 == 0:
                torch.cuda.empty_cache()

    pd.DataFrame(results).to_csv(CFG["csv_path"], index=False)
    display(FileLink(CFG["model_path"]))

if __name__ == "__main__":
    main()

In [ ]:
import os
import cv2
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import segmentation_models_pytorch as smp
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm

# --- CONFIG ---
MODEL_PATH = "best_rugged_ultra.pth"
# Yahan TEST images ka path dalein
TEST_IMG_DIR = "/kaggle/input/terra-seg-rugged-terrain-segmentation/offroad-seg-kaggle/test_images_padded" 
OUT_CSV = "submission.csv"
VISUAL_OUT = "prediction_check.png" # Visual results yahan save honge
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
H, W = 540, 960  # Kaggle expected resolution (Submission size)

def rle_encode(mask01):
    """Kaggle specific RLE encoding (column-major)."""
    pixels = mask01.flatten(order="F")
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(map(str, runs))

def generate_submission():
    # 1. Model Setup
    model = smp.UnetPlusPlus(
        encoder_name="efficientnet-b6", 
        encoder_weights=None, 
        in_channels=3, 
        classes=1
    ).to(DEVICE)
    
    if not os.path.exists(MODEL_PATH):
        print(f"❌ Error: Model file {MODEL_PATH} nahi mili!")
        return

    print(f"🔄 Loading model from {MODEL_PATH}...")
    checkpoint = torch.load(MODEL_PATH, map_location=DEVICE)
    state_dict = checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint
    model.load_state_dict(state_dict)
    model.eval()

    # 2. Transform Setup (Inference resize 768)
    transform = A.Compose([
        A.Resize(768, 768), 
        A.Normalize(), 
        ToTensorV2()
    ])
    
    if not os.path.exists(TEST_IMG_DIR):
        print(f"❌ Error: Test image directory {TEST_IMG_DIR} nahi mili!")
        return
        
    all_imgs = sorted([f for f in os.listdir(TEST_IMG_DIR) if f.endswith(('.png', '.jpg'))])
    print(f"🚀 Found {len(all_imgs)} images. Starting Inference...")

    rows = []
    visual_samples = [] # Visual results store karne ke liye

    # 4. Inference Loop
    with torch.no_grad():
        for i, name in enumerate(tqdm(all_imgs)):
            img_path = os.path.join(TEST_IMG_DIR, name)
            
            # Load Image
            img_bgr = cv2.imread(img_path)
            if img_bgr is None: continue
            img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
            
            # Preprocess
            input_tensor = transform(image=img_rgb)["image"].unsqueeze(0).to(DEVICE)
            
            # Predict
            logits = model(input_tensor)
            probs = torch.sigmoid(logits).detach().cpu().numpy()[0, 0]
            
            # Resize back to Submission size
            probs_resized = cv2.resize(probs, (W, H), interpolation=cv2.INTER_LINEAR)
            mask01 = (probs_resized > 0.5).astype(np.uint8)
            
            # RLE Encode
            rle = rle_encode(mask01)
            image_id = os.path.splitext(name)[0]
            rows.append({"image_id": image_id, "encoded_pixels": rle})

            # Visual check ke liye pehli 3 images save karein
            if i < 3:
                visual_samples.append((img_rgb, mask01, name))

    # 5. Visual Result Save Karein (Matplotlib)
    if visual_samples:
        print(f"🖼️ Saving visual samples to {VISUAL_OUT}...")
        plt.figure(figsize=(15, 5 * len(visual_samples)))
        for idx, (original, pred, title) in enumerate(visual_samples):
            # Original Image
            plt.subplot(len(visual_samples), 2, idx*2 + 1)
            plt.imshow(original)
            plt.title(f"Original: {title}")
            plt.axis('off')
            
            # Predicted Mask
            plt.subplot(len(visual_samples), 2, idx*2 + 2)
            plt.imshow(pred, cmap='gray')
            plt.title("Model Prediction (Terrain)")
            plt.axis('off')
        
        plt.tight_layout()
        plt.savefig(VISUAL_OUT)
        plt.close()

    # 6. CSV Save Karein
    df = pd.DataFrame(rows)
    df.to_csv(OUT_CSV, index=False)
    
    print(f"\n✅ Success!")
    print(f"📂 CSV Saved: {OUT_CSV}")
    print(f"🖼️ Visual Check Saved: {VISUAL_OUT}")
    print(df.head())

if __name__ == "__main__":
    generate_submission()